# threading

### Процесс и Поток

Процесс - программа, которая выполняется в текущий момент.

<Представление программы в памяти>: https://commons.wikimedia.org/wiki/File:C-memlayout-ru.svg?uselang=ru

Поток (поток выполнения, thread) - единица обработки, исполнение которой может быть назначено ядром операционной системы. Исполняющаяся последовательность инструкций внутри процесса.

Поток также называют "легковестным процессом" (https://learn-gevent-socketio.readthedocs.io/en/latest/general_concepts.html).

Несколько потоков выполнения могут существовать в рамках одного и того же процесса и совместно использовать его ресурсы.

Процесс с двумя потоками выполнения на одном ядре процессора: https://commons.wikimedia.org/wiki/File:Multithreaded_process.svg?uselang=ru

Процессы с тредами: https://sites.google.com/site/sureshdevang/thread-vs-process


* одно ядро процессора в один момент времени может исполнять только один тред
* треды одного процесса могут исполняться физически одновременно (на разных ядрах)
* бессмысленно порождать вычислительных тредов больше, чем у вас есть ядер

### Исполнение кода питона

CPython (самая популярная реализация интерпретатора питона) был реализован с максимальной простотой и имеет потокобезопасный механизм - GIL (Global Interpreter Lock).

Благодаря этому Lock'у интерпретатор питона может исполнять лишь одну команду в один момент времени (single threading). По этой причине, создание несколько потоков не приведет к их одновременному исполнению на разных ядрах процессора, тем не менее, потоки полезны и в python.

In [ ]:
# модуль питона для работы с потоками
import threading

Рассмотрим простой пример программы, создающей потоки:

In [23]:
import threading # модуль для работы с потоками (threads)
import sys

def thread_job(number):
    print('Hello {}'.format(number))
    sys.stdout.flush()

def run_threads(count):
    thread_job(0)
    threads = [
        threading.Thread(target=thread_job, args=(i,))
        for i in range(1, count + 1)
    ]
    for thread in threads:
        thread.start()  # каждый поток должен быть запущен
    for thread in threads:
        thread.join()  # дожидаемся исполнения всех потоков

run_threads(4)

Hello 0
Hello 1
Hello 2Hello 3Hello 4




### Упражнение №1

Запустите следующий код. В чем проблема данного кода? Всегда ли counter = 10 после исполнения кода программы?

In [26]:
counter = 0

def thread_job():
    global counter
    old_counter = counter
    counter = old_counter + 1
    print('{} '.format(counter), end='')

threads = [threading.Thread(target=thread_job) for _ in range(10)]
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()

counter

2 1 3 4 5 6 7 8 9 10 

10

Демонстрация "проблемности" кода:

In [33]:
import random
import time

counter = 0
def thread_job():
    global counter
    old_counter = counter
    time.sleep(random.randint(0, 1))
    counter = old_counter + 1
    print('{} '.format(counter), end='')

threads = [threading.Thread(target=thread_job) for _ in range(10)]
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()
counter

1 2 3 4 5 6 7 8 9 10 

10

#### Почему так происходит?

Одно из возможных решений (не самое аккуратное):

In [21]:
counter = 0

def thread_job(lock):
    lock.acquire() # mutex
    global counter
    counter += 1
    print('{} '.format(counter), end='')
    sys.stdout.flush()
    lock.release()

lock = threading.Lock()
threads = [
    threading.Thread(target=thread_job, args=(lock,))
    for i in range(10)
]
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()

counter

1 2 3 4 5 6 7 8 9 10 

10

Решение лучше (с with, https://jeffknupp.com/blog/2016/03/07/python-with-context-managers):

In [30]:
counter = 0

def thread_job(lock):
    with lock:
        global counter
        counter += 1
        print('{} '.format(counter), end='')
        sys.stdout.flush()

lock = threading.Lock()
threads = [
    threading.Thread(target=thread_job, args=(lock,))
    for i in range(10)
]
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()

counter

1 2 3 4 5 6 7 8 9 10 

10

Лучшее решение (используя queue (очереди) на счет и вывод на экран):

In [34]:
import threading
import queue

class Counter:
    def __init__(self, value):
        self.value = value

def printer(printing_queue):
    while True:
        value = printing_queue.get()
        print(value)
        printing_queue.task_done()

def calculator(counter, calculation_queue, printing_queue):
    while True:
        delta = calculation_queue.get()
        counter.value += delta
        printing_queue.put(counter.value)
        calculation_queue.task_done()

def delta_generator(calculation_queue):
    calculation_queue.put(1)

# Main
printing_queue = queue.Queue()
printer_daemon = threading.Thread(
    target=printer,
    args=(printing_queue,),
    daemon=True
)
printer_daemon.start()

counter = Counter(0)
calculation_queue = queue.Queue()
calculator_daemon = threading.Thread(
    target=calculator,
    args=(counter, calculation_queue, printing_queue),
    daemon=True
)
calculator_daemon.start()

workers = [
    threading.Thread(target=delta_generator, args=(calculation_queue,))
    for _ in range(10)
]
for worker in workers:
    worker.start()
for worker in workers:
    worker.join()

calculation_queue.join()
printing_queue.join()


1
2
3
4
5
6
7
8
9
10


* ошибки в многопоточном коде - одни из самых неприятных
* модуль queue позволяет на порядок меньше думать и ошибаться, это самый pythonic способ писать многопоточный код

### Упражнение:
Написать программу, которая будет находить сумму чисел массива с использованием N тредов
(N <= core_count)

Пример:

In [36]:
import queue

def thread_job(arr, part_id, thread_count, results_queue):
    results_queue.put(
        sum(arr[i] for i in range(part_id, len(arr), thread_count))
    )

def sum_using_threads(arr, thread_count):
    results_queue = queue.Queue()
    threads = [
        threading.Thread(target=thread_job, args=(arr, i, thread_count, results_queue))
        for i in range(thread_count)
    ]
    for thread in threads:
        thread.start()

    results = []
    for thread in threads:
        results.append(results_queue.get())
        thread.join()

    return sum(results)

In [37]:
arr = [1 for _ in range(10 * 1000 * 1000)]

In [40]:
%%timeit
sum_using_threads(arr, 1)

912 ms ± 28.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [39]:
%%timeit
sum_using_threads(arr, 4)

905 ms ± 22.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Видим влияние GIL (Lock) на исполнение. **Вычисления** распараллеливать бессмысленно

Существует сценарии, при которых использование потоков оправдано!

Упражнение №.:

In [46]:
import urllib.request

urls = [
    'https://www.yandex.ru', 'https://www.google.com',
    'https://habrahabr.ru', 'https://www.python.org',
    'https://isocpp.org',
]

def read_url(url):
    with urllib.request.urlopen(url) as u:
        return u.read()

In [43]:
%%timeit
for url in urls:
    read_url(url)

6.18 s ± 341 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
%%timeit
readers = [
    threading.Thread(target=read_url, args=(url,))
    for url in urls
]
for reader in readers:
    reader.start()
for reader in readers:
    reader.join()

2.28 s ± 181 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# multiprocessing

# asyncio